In [ ]:
from BayesianHoldem import BayesianHoldem
import torch

model = BayesianHoldem()

for i in range(10):
    card_representation = torch.randn(6, 13, 4)
    action_representation = torch.randn(24, 4, 4)
    prediction = model.predict_action(action_representation, card_representation)
    if prediction == 0:
        print(f"prediction {i}: Fold")
    elif prediction == 1:
        print(f"prediction {i}: Check/Call")
    elif prediction == 2:
        print(f"prediction {i}: Bet/Raise (BB)")
    elif prediction == 3:
        print(f"prediction {i}: All-in")

In [8]:
import pokerkit as pk

big_blind = 100
small_blind = 0.5*big_blind
blinds = [small_blind, big_blind]
ante = 0
min_bet = 0.5*big_blind
player_count = 2

P1_stack = 100*big_blind
P2_stack = 50*big_blind

game = pk.NoLimitTexasHoldem(
    # Automations - everything is automatic except for player actions
    (
        pk.Automation.ANTE_POSTING,
        pk.Automation.BET_COLLECTION,
        pk.Automation.BLIND_OR_STRADDLE_POSTING,
        pk.Automation.CARD_BURNING,
        pk.Automation.HOLE_DEALING,
        pk.Automation.BOARD_DEALING,
        pk.Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        pk.Automation.HAND_KILLING,
        pk.Automation.CHIPS_PUSHING,
        pk.Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True, # irrelevant but necessary for the game to be created
    raw_antes=ante,
    raw_blinds_or_straddles=blinds,
    min_bet=min_bet,
)

state = game(raw_starting_stacks=[P1_stack, P2_stack], player_count=player_count)
print(type(state))
print(f"Street index: {state.street_index}")
print(f"Stacks: {state.stacks}")

<class 'pokerkit.state.State'>
Street index: 0
Stacks: [9900, 4950.0]


In [158]:
import GameRepresentations
import importlib
importlib.reload(GameRepresentations)

raise_amount = 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, torch.zeros((24, 4, 4)))
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
raise_amount += 150
state.complete_bet_or_raise_to(raise_amount)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
print(action_tensor)

tensor([[[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[

In [ ]:
import GameRepresentations
import importlib
importlib.reload(GameRepresentations)

state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, torch.zeros((24, 4, 4)))
state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.complete_bet_or_raise_to(150)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.complete_bet_or_raise_to(300)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.complete_bet_or_raise_to(450)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
state.complete_bet_or_raise_to(state.stacks[state.actor_index] + state.bets[state.actor_index])
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)
print(action_tensor)


tensor([[[0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 2., 0., 0.],
         [0., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 2., 0.],
         [1., 1., 1., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 1., 1., 1.]],

        [[

In [ ]:

import GameRepresentations
import importlib
importlib.reload(GameRepresentations)

state.check_or_call()
state.complete_bet_or_raise_to(150)
print(state.street_index)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, torch.zeros((24, 4, 4)))

state.complete_bet_or_raise_to(200)
state.complete_bet_or_raise_to(250)
print(state.street_index)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)

state.check_or_call()
state.complete_bet_or_raise_to(150)
state.check_or_call()
print(state.street_index)
action_tensor = GameRepresentations.GameRepresentations.get_action_representations(state, action_tensor)

print(action_tensor)

In [11]:
import GameRepresentations
import importlib
importlib.reload(GameRepresentations)
state.check_or_call()
state.check_or_call()
action_tensor = GameRepresentations.GameRepresentations.get_card_representations(state, 0)
print(action_tensor)
print(state.hole_cards[0])
print(state.board_cards)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
print(type(state.operations[-1]))
state.check_or_call()
state.check_or_call()
print(type(state.operations[-1]))
state.complete_bet_or_raise_to(500)
state.fold()
print(type(state.operations[-1]))
print(tuple(state.pot_amounts)[0])
print(state.board_cards)
print(state.hole_cards)
print(state.operations)

In [ ]:
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")


state.complete_bet_or_raise_to(state.stacks[state.actor_index] + state.bets[state.actor_index])
print(f"All in")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

state.check_or_call()
print("Check or call")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

print(f"Street index: {state.street_index}")

In [ ]:
# Test self-play

from SelfPlayPokerGame import SelfPlayPokerGame

game = SelfPlayPokerGame()

for i in range(10):
    print(f"===============")
    print(f" New Game {i+1}")
    print("===============")
    game.run_game(verbose=True, training=False)

In [1]:
# Test training
import SelfPlayPokerGame
import importlib
importlib.reload(SelfPlayPokerGame)

from SelfPlayPokerGame import SelfPlayPokerGame
import os

game = SelfPlayPokerGame()

save_path = "data/models/test"
os.makedirs(save_path, exist_ok=True)

training_stats = game.train(num_sessions=15, games_per_session=30, save_path=save_path)


========= Starting Training Session 1/15 =========
========= New game 1 =========
========= New game 2 =========
========= New game 3 =========
========= New game 4 =========
========= New game 5 =========
========= New game 6 =========
========= New game 7 =========
========= New game 8 =========
========= New game 9 =========
========= New game 10 =========
========= New game 11 =========
========= New game 12 =========
========= New game 13 =========
========= New game 14 =========
========= New game 15 =========
========= New game 16 =========
========= New game 17 =========
========= New game 18 =========
========= New game 19 =========
========= New game 20 =========
========= New game 21 =========
========= New game 22 =========
========= New game 23 =========
========= New game 24 =========
========= New game 25 =========
========= New game 26 =========
========= New game 27 =========
========= New game 28 =========
========= New game 29 =========
========= New game 30 =======

In [ ]:
from importlib import reload

import utilities
reload(utilities)
from utilities import Utilities


Utilities.plot_all_stats(training_stats, save_dir=save_path)

In [ ]:
print(training_stats)